In [8]:
import igl 
import meshplot as mp
import numpy as np
import polyfempy as pf
import wildmeshing as wm

In [11]:
v, f = igl.read_triangle_mesh("ressources/3way/tri1.obj")
#it is the colorcoding because the faces on the top of the sphere are more bright 
#, which match with the y-values of their coordinates are the biggest at the top.
mp.plot(v, f, v[:,1], shading={"wireframe": True})
print(v)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(10.012492…

[[ 10.01249218   0.         -17.34214592]
 [  7.50936937   0.         -13.00660896]
 [  5.00624609   0.          -8.67107296]
 [  6.64022064   0.          -7.72048473]
 [  5.64646673   0.          -5.99925232]
 [  7.37851763   0.          -4.99925232]
 [  8.37539482   0.          -6.72589397]
 [ 10.01249218   0.          -5.78071499]
 [ 11.4820509    0.          -6.62916517]
 [ 12.4820509    0.          -4.89711428]
 [ 14.20785522   0.          -5.90793276]
 [ 13.21410179   0.          -7.62916517]
 [ 15.01873875   0.          -8.67107296]
 [ 12.51561546   0.         -13.00660896]]


In [3]:
vc, fc = igl.read_triangle_mesh("ressources/sphere_with_hole.obj")
# Find the open boundary
bnd = igl.boundary_loop(fc)
# Map the boundary to a circle, preserving edge proportions
bnd_uv = igl.map_vertices_to_circle(vc, bnd)
# Harmonic parametrization for the internal vertices
uv = igl.harmonic_weights(vc, fc, bnd, bnd_uv, 1)
print(bnd)
# Plotting the results
p = mp.subplot(vc, fc, uv=uv, s=[1, 2, 0])
mp.subplot(uv, fc, uv=uv, shading={"wireframe": True, "wire_color": "red", "wire_width": 1.0}, s=[1, 2, 1], data=p)

# Adding the boundary
p.rows[0][0].add_points(vc[bnd], shading={"point_size": 0.1});
lines = np.vstack([bnd, np.roll(bnd, -1)]).T
p.rows[0][0].add_edges(vc, lines, shading={"line_width": 1.0, "line_color": "red"});

[ 58  59  74  89 104 103  88  73]


# MeshPlot
    plot a mesh by its vertices and faces:
        mp.plot(v, f,v[:, 2], shading={"wireframe": True}) ???(the 2)
        
    plot the triangulated svg file with wildmeshing:
        p = mp.plot(vs, tris, shading={"wireframe": True}, return_plot=True)
        
    add vertices points and nodes ??? points:
        p.add_points(V, shading={"point_color": "red", "point_size": 2});
        p.add_points(nodes, shading={"point_color": "green", "point_size": 2})
 **here is not adding the nodes, it just add point named "nodes" to render**
        
    plot the ployfem solved problem: 
        mp.plot(vertices, tets, mises, shading={"wireframe": False}, return_plot=True)
    


# IGL
    read_triangle_mesh:       input: XXX.obj 
                              output: (v,f) vertices, faces
                        
    bboundary_loop:           input: faces 
                              output: np.array of the missing loops
                    
    map_vertices_to_circle:   input: (vertices, boundary_loop) 
                              output: 2d circle mapping of the boundary 
                            
    harmonic_weights:         input: (vertices, faces, boundary_loop, boundary 2d circle map, 1 (1 for harmonic function, 2 for biharmonic etc ) 
                              output: 2D coordinates of the flattened mesh as two harmonic functions (np.array)
    
    
    
# WildMeshing
    tetrahedralize:           input: (source.obj, dest.mesh)
                              output: none
                              
    triangulate_svg:          input: (svg_path,
                                    stop_quality = -1,
                                    max_its = 80,
                                    stage = 1,
                                    epsilon = -1,
                                    feature_epsilon = 1e-3,
                                    target_edge_len = -1,
                                    edge_length_r = 1./20.,
                                    flat_feature_angle = 10.,
                                    cut_outside = False,
                                    skip_eps = False,
                                    hole_pts = None,
                                    mute_log = False)
                                    
                              output: Vertices_out, Faces_out, nodes, Faces_nodes 
                             
              
# Polyfempy 
    settings:                input: none
                             output: settings to ajust for the problem solver
                  
          settings.set_pde(any PDE) ex: polyfempy.PDEs.LinearElasticity
          settings.discr_order = 2 order of the PDE
          settings.set_material_params("E", 210000) Young’s modulus and poisson ratio
          settings.set_material_params("nu", 0.3)
          settings.set_problem(problem) setting up the problem
          
    problem:                 input: none
                             output: the setup problem 
          problem.set_x_symmetric(n) set the stress symmetric boundary condition on axe x
          problem.set_force(a, [100, 0]) set a force vector [100,0] on sideset a
          
    solver: 
          solver = pf.Solver()
          solver.settings(settings) setup the settings for the solver
          solver.load_mesh_from_path(mesh_path) load the mesh
          solver.solve() solve
          solver.get_sampled_solution(): output: [pts, tets, disp] pts the points, disp the displacement, tets ???
          solver.get_sampled_mises_avg(): output: (mises, _ ) mises is the stresses applied to the mesh

In [4]:
wm.tetrahedralize("ressources/bunny.obj", "ressources/bunny.mesh", mute_log=True)

In [5]:
vertices = np.array([[0,0],[1,0], [1,1], [0,1]])
edges = np.array([[0,1],[1,2],[2,3],[0,3]])
faces = np.array([[0,1,2]])

In [6]:
p=mp.plot(vertices,faces, return_plot=True)
p.add_edges(vertices ,edges, shading={});
p.add_points(vertices, shading={"point_color": "red", "point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…

2

In [7]:
vs, tris, nodes, F_nodes = wm.triangulate_svg("figure[1].svg", cut_outside=True)
p = mp.plot(vs, tris, shading={"wireframe": True}, return_plot=True)
V,L = igl.read_triangle_mesh("ressources/plane_hole.obj") #??? how do i get the edges instead of faces

FileNotFoundError: [Errno 2] No such file or directory: 'figure[1].svg'

In [ ]:
p=mp.plot(V, np.zeros((0,3)), return_plot=True)
p.add_edges(V, L, shading={}); 
p.add_points(V, shading={"point_color": "red", "point_size": 0.1})

In [ ]:
import tetgen
import pyvista as pv
sphere = pv.Sphere()
tet = tetgen.TetGen(sphere)
tet.tetrahedralize(order=1, mindihedral=3, minratio=1)
grid = tet.grid
grid.plot(show_edges=True)
cells = grid.cells.reshape(-1, 5)[:, 1:]
cell_center = grid.points[cells].mean(1)

mask = cell_center[:, 2] < 0.2
cell_ind = mask.nonzero()[0]
subgrid = grid.extract_cells(cell_ind)

plotter = pv.Plotter()
plotter.add_mesh(subgrid, 'lightgrey', lighting=True, show_edges=True)
plotter.add_mesh(sphere, 'r', 'wireframe')
plotter.add_legend([[' Input Mesh ', 'r'],
                    [' Tesselated Mesh ', 'black']])
plotter.show()